In [7]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import cv2

In [59]:
img = cv2.imread("/home/roger/Desktop/eye.jpg")

lfilter = [[0, 0, 0],
           [0.3, 0.3, 0.3],
           [0, 0, 0]]
lfilter = np.array(lfilter)

def add_padding(matrix, kernel, pad):
    """ add padding to the matrix """
    for p in range(pad):
        col_zeros = np.array([0]*matrix.shape[0])
        matrix = np.c_[col_zeros, matrix, col_zeros]
    for p in range(pad):
        row_zeros = np.array([0]*matrix.shape[1])
        matrix = np.r_[[row_zeros], matrix, [row_zeros]]
    return matrix


def register(matrix, r, c, pad):
    """ get the neighbors of a certain matrix """
    lower_r = r-pad
    upper_r = r+pad+1
    lower_c = c-pad
    upper_c = c+pad+1
    return matrix[lower_r:upper_r,lower_c:upper_c]


def correlate(matrix, kernel):
    """ apply the filter on the matrix """
    mult = np.multiply(matrix, kernel)
    return np.sum(mult)


def convolve(matrix, kernel):
    """ apply convolutional filter on the matrix"""
    kernel = np.flipud(np.fliplr(kernel))
    mult = np.multiply(matrix, kernel)
    return np.sum(mult)
    
    
def apply_filter(img, kernel):
    imgout = np.zeros(img.shape)
    pad = (kernel.shape[0]-1)/2
    nb_channels = img.shape[2]
    for ch in range(nb_channels):
        mat = img[:,:,ch]
        matpad = add_padding(mat, kernel, pad)
        
        nr, nc = 0, 0
        for (r, c), value in np.ndenumerate(matpad):
            if r >= pad and r < matpad.shape[1] - pad \
               and c >= pad and c < matpad.shape[0] - pad:
                matreg = register(matpad, r, c, pad)
                pixelconv = convolve(matreg, kernel)
                imgout[r-pad][c-pad] = pixelconv
    return imgout
        
imgout = apply_filter(img, lfilter)
cv2.imwrite('/home/roger/Desktop/imgout.jpg', imgout)

True

In [49]:
resized_image = cv2.resize(img, (64, 64)) 
cv2.imwrite('/home/roger/Desktop/eye64.jpg', resized_image)

True